In [14]:
import os

from dotenv import load_dotenv

load_dotenv()

ID = os.environ.get("ID")
PW = os.environ.get("PW")

테스트 한다고 계속 돌리다가 clipboard를 이용한 우회 방법 또한 CAPTCHA가 발생할 수 있는데  
이는 여러 번 로그인을 시도한 케이스라 `실제로 로그인을 한 번 진행` 후 하면 잘 우회된다.

In [34]:
import time

import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


NAVER_LOGIN_URL = "https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/"

driver =webdriver.Chrome()
driver.get(NAVER_LOGIN_URL)

# id 입력
id_element = driver.find_element(By.CSS_SELECTOR, "#id")
pyperclip.copy(ID)
id_element.send_keys(Keys.COMMAND, 'v')
time.sleep(1)

# pw 입력
pw_element = driver.find_element(By.CSS_SELECTOR, "#pw")
pyperclip.copy(PW)
pw_element.send_keys(Keys.COMMAND, 'v')
time.sleep(1)

# trigger login submit
pw_element.send_keys(Keys.ENTER)



# 메일 함 이동 & 창 전환

사실 그냥 메일 페이지 url을 통해서 들어가는 것이 간편하지만 창전환 연습을 위해 구현

In [35]:
mail_button = driver.find_element(By.CSS_SELECTOR, "#account > div.MyView-module__my_menu___eF24q > div > div > ul > li:nth-child(1) > a > span.MyView-module__item_text___VTQQM")
mail_button.click()

time.sleep(1)

link_to_mail_page = driver.find_element(By.CSS_SELECTOR, "#account > div.MyView-module__layer_menu_service___NqMyX > div.MyView-module__service_sub___wix9p > div.MyView-module__sub_left___AIWHR > a")
link_to_mail_page.click()

# 창 전환(메일 페이지)
driver.switch_to.window(driver.window_handles[1])

# 메일 쓰기

In [36]:
write_main_button = driver.find_element(By.CSS_SELECTOR, "#root > div > nav > div > div.lnb_header > div.lnb_task > a.item.button_write")
write_main_button.click()

In [37]:
time.sleep(1)
receiver_input = driver.find_element(By.CSS_SELECTOR, "#recipient_input_element")
receiver_input.send_keys("rlwjd31@naver.com")

time.sleep(1)
email_title_input = driver.find_element(By.CSS_SELECTOR, "#subject_title")
email_title_input.send_keys("테스트 메일")


# 이메일 본문 입력란 iframe 접근

iframe은 page 내부의 다른 page이기 driver의 제어권을 iframe으로 전환하고 작업이 끝나면 다시 돌아와야 한다.

- default_content: iframe안에 iframe이 있다면 상위 iframe으로 돌아가는 줄 알았는데 그냥 root page로 돌아가는 것 같다.

In [38]:
email_content_iframe = driver.find_element(By.CSS_SELECTOR, "#content > div.contents_area > div > div.editor_area > div > div.editor_body > iframe")
driver.switch_to.frame(email_content_iframe)

content_input = driver.find_element(By.CSS_SELECTOR, "body > div > div.workseditor-body > div.workseditor-content")
content_input.send_keys("테스트 메일 본문")

# 이메일 본문 입력란 iframe 접근 해제
# default_
driver.switch_to.default_content()




# 메세지 보내기

`내게 쓰기`를 통해서 메일을 보내지 않고 `메일 쓰기`로 보내면 왜 휴지통에 들어가있음......? ㅜㅜ

In [39]:
time.sleep(1)
send_button = driver.find_element(By.CSS_SELECTOR, "#content > div.mail_toolbar.type_write > div:nth-child(1) > div > button.button_write_task")
send_button.click()

# 전체 코드 & 리팩터링

In [ ]:
from dataclasses import dataclass
import os
from time import sleep

import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from dotenv import load_dotenv

load_dotenv()

ID = os.environ.get("ID")
PW = os.environ.get("PW")
NAVER_LOGIN_URL = "https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/"


driver =webdriver.Chrome()
driver.get(NAVER_LOGIN_URL)

# =========== 네이버 로그인 ===========
# id 입력
id_element = driver.find_element(By.CSS_SELECTOR, "#id")
pyperclip.copy(ID)
id_element.send_keys(Keys.COMMAND, 'v')
sleep(1)

# pw 입력
pw_element = driver.find_element(By.CSS_SELECTOR, "#pw")
pyperclip.copy(PW)
pw_element.send_keys(Keys.COMMAND, 'v')
sleep(1)

# trigger login submit
pw_element.send_keys(Keys.ENTER)
# ============================================


# =========== 메일함 이동 & 창 전환 ===========
sleep(1)
mail_button = driver.find_element(By.CSS_SELECTOR, "#account > div.MyView-module__my_menu___eF24q > div > div > ul > li:nth-child(1) > a > span.MyView-module__item_text___VTQQM")
mail_button.click()

sleep(1)

link_to_mail_page = driver.find_element(By.CSS_SELECTOR, "#account > div.MyView-module__layer_menu_service___NqMyX > div.MyView-module__service_sub___wix9p > div.MyView-module__sub_left___AIWHR > a")
link_to_mail_page.click()

# 창 전환(메일 페이지)
driver.switch_to.window(driver.window_handles[1])
# ============================================

# ================== 메일쓰기 ==================
# 메일 쓰기 버튼 클릭
sleep(1)
write_main_button = driver.find_element(By.CSS_SELECTOR, "#root > div > nav > div > div.lnb_header > div.lnb_task > a.item.button_write")
write_main_button.click()

# 받는 사람 입력
sleep(1)
receiver_input = driver.find_element(By.CSS_SELECTOR, "#recipient_input_element")
receiver_input.send_keys("rlwjd31@naver.com")

# 이메일 제목 입력
sleep(1)
email_title_input = driver.find_element(By.CSS_SELECTOR, "#subject_title")
email_title_input.send_keys("테스트 메일")

# 이메일 본문 iframe 접근
email_content_iframe = driver.find_element(By.CSS_SELECTOR, "#content > div.contents_area > div > div.editor_area > div > div.editor_body > iframe")
driver.switch_to.frame(email_content_iframe)

content_input = driver.find_element(By.CSS_SELECTOR, "body > div > div.workseditor-body > div.workseditor-content")
content_input.send_keys("테스트 메일 본문")

# 이메일 본문 입력란 iframe 접근 해제
driver.switch_to.default_content()
# ============================================

# ================== 메일쓰기 ==================
sleep(1)
send_button = driver.find_element(By.CSS_SELECTOR, "#content > div.mail_toolbar.type_write > div:nth-child(1) > div > button.button_write_task")
send_button.click()
# ============================================





In [51]:
driver.quit()

In [16]:
driver.quit()

# pip install pypeclip